# 라이브러리 설치

In [ ]:
!pip install -q chromadb sentence-transformers openai ipywidgets
!jupyter nbextension enable --py widgetsnbextension

# API Key 환경 설정

In [11]:
from google.colab import userdata
HF_TOKEN = userdata.get('HF_TOKEN') # Hugging Face Token
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY') # Google Custom Search API 키
GOOGLE_CSE_ID = userdata.get('GOOGLE_CSE_ID') # Programmable Search Engine ID
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY') # OpenAI API Key
client_id = userdata.get('NAVER_CLIENT_ID') # Naver Client Key
client_secret = userdata.get('NAVER_CLIENT_SECRET') # Naver Client Secret

# Base Knowledge 수집용 네이버쇼핑 영양제 상품 검색 코드

In [9]:
import requests
from urllib.parse import quote

def naver_shop_search(query, display=100):
    encoded_query = quote(query)
    url = f"https://openapi.naver.com/v1/search/shop.json?query={encoded_query}&display={display}&start=1&sort=sim&exclude=used:cbshop:rental"

    headers = {
        "X-Naver-Client-Id": client_id,
        "X-Naver-Client-Secret": client_secret
    }

    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        return response.json()
    else:
        return {"error": response.status_code}

result = naver_shop_search("영양제", 100)

# 추가 정보 수집을 위한 구글 검색 코드

In [8]:
from googleapiclient.discovery import build

def google_search(query, num=10):
    service = build("customsearch", "v1", developerKey=GOOGLE_API_KEY)
    res = service.cse().list(q=query, cx=GOOGLE_CSE_ID, num=num).execute()
    return [item["snippet"] for item in res["items"]]


# Base Knowledge를 Vector DB에 저장하는 코드

In [12]:
# 네이버쇼핑 검색 결과 -> Vector DB (Chroma) 저장
import chromadb
from chromadb.utils import embedding_functions
from sentence_transformers import SentenceTransformer

# 데이터 전처리
def preprocess_items(items):
    processed = []
    for item in items:
        data = {
            "id": item['productId'],
            "text": f"{item['title']} {item['brand']} {item['maker']} {item['category3']}",  # 임베딩용 텍스트
            "metadata": {
                "price": int(item['lprice']),
                "brand": item['brand'],
                "category": f"{item['category1']}>{item['category2']}>{item['category3']}",
                "link": item['link']
            }
        }
        processed.append(data)
    return processed

# Chroma DB 초기화
client = chromadb.PersistentClient(path="./naver_shopping_db")
embedding_model = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name="sentence-transformers/paraphrase-multilingual-mpnet-base-v2", token=HF_TOKEN
)
# 기존 컬렉션이 있으면 삭제 후 재생성
try:
    client.delete_collection("products")
except:
    pass
collection = client.create_collection(
    name="products",
    embedding_function=embedding_model,
    metadata={"hnsw:space": "cosine"}
)

# 데이터 저장
if 'items' in result:
    processed_data = preprocess_items(result['items'])
    collection.add(
        ids=[item['id'] for item in processed_data],
        documents=[item['text'] for item in processed_data],
        metadatas=[item['metadata'] for item in processed_data]
    )
    print("성공적으로", len(processed_data), "개 상품 저장됨")
else:
    print("Error:", result.get('error', '데이터 없음'))


성공적으로 100 개 상품 저장됨


# Vector DB 검색 코드

In [ ]:
def vectordb_search(query, n_results=10):
    results = collection.query(
        query_texts=[query],
        n_results=n_results
    )
    output = []
    for idx, (doc, meta) in enumerate(zip(results['documents'][0], results['metadatas'][0]), 1):
        output.append({
            "제품정보": doc,
            "메타데이터": meta
        })
    return output
vectordb_search("루테인")

# 검색 도구 함수 정의 (Function Calling)

In [19]:
recommend_nutrient = {
    "name": "recommend_nutrient",
    "description": "사용자가 입력한 상태를 분석해 영양소를 추천합니다.",
    "parameters": {
        "type": "object",
        "properties": {
            "nutrient": {"type": "string", "description": "추천하는 영양소 성분"},
            "query": {"type": "string", "description": "사용자 상태 쿼리"
            }
        },
        "required": ["nutrient", "query"]
    }
}

excessive_nutrient = {
    "name": "excessive_nutrient",
    "description": "사용자가 입력한 영양제 섭취 상태를 분석해 일일 권장량을 초과하는 영양소를 찾습니다.",
    "parameters": {
        "type": "object",
        "properties": {
            "nutrient": {"type": "string", "description": "초과하는 영양소 성분"},
            "query": {"type": "string", "description": "계산 수식과 초과한 이유"
            }
        },
        "required": ["nutrient", "query"]
    }
}

vectordb_search_schema = {
  "name": "vectordb_search",
  "description": "추천 영양소로 Vector DB에서 제품 검색",
  "parameters": {
    "type": "object",
    "properties": {
      "query": {"type": "string"},
      "max_results": {"type": "integer", "default": 10}
    },
    "required": ["query"]
  }
}

google_search_function_schema = {
  "name": "google_search",
  "description": "검색된 제품을 Google 검색해서 상세 정보 조회",
  "parameters": {
    "type": "object",
    "properties": {
      "query": {"type": "string"},
      "num_results": {"type": "integer", "default": 10}
    },
    "required": ["query"]
  }
}


# LLM 동작 코드

In [23]:
from openai import OpenAI
import json
import re

client = OpenAI(api_key=userdata.get('OPENAI_API_KEY'))

def nutrient_too_much_check(symptom, gender, age, pregnancy, mode):
    prompt = f"""
      사용자 정보:
      - 성별: {gender}
      - 나이: {age}
      - 임신 여부: {pregnancy}
      - 원하는 것: {mode}
      - 상태: {symptom}
  현재 사용자가 섭취하고 있는 상태인 각각 영양소의 총 함량과 해당 영양소의 일일 권장량을 비교하여 초과/미달 여부를 계산하세요. 계산 근거를 같이 제시하세요.
      """
    messages = [
        {
            "role": "system",
            "content": "너는 건강 기반의 영양제 조언 전문가야. 전문적으로 답변해줘."
        },
        {"role": "user", "content": prompt}
    ]

    # Step 1
    print("현재 messages:", messages)
    res = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        functions= [excessive_nutrient],
        function_call={"name": "excessive_nutrient"},
        temperature=0.5
        )
    # print("LLM 전체 응답:", res)
    func_name = res.choices[0].message.function_call.name
    print("호출된 함수:", func_name)
    print("함수 인자(raw):", res.choices[0].message.function_call.arguments)

    args = json.loads(res.choices[0].message.function_call.arguments)
    print(args)
    # nutrient = args["nutrient"]
    # print(f"추천 영양소: {nutrient}")


def nutrient_recommend_check(symptom, gender, age, pregnancy, mode):
  prompt = f"""
    사용자 정보:
    - 성별: {gender}
    - 나이: {age}
    - 임신 여부: {pregnancy}
    - 원하는 것: {mode}
    - 상태: {symptom}

    """

  if mode=="식습관 분석을 통한 영양제 추천":
    prompt+="\n현재 사용자의 식사 습관 상태를 고려하여 결핍되거나 보조적으로 섭취가 필요한 가장 적절한 영양소를 1가지 추천하세요.\n"
    nutrient_recommend(prompt,symptom, gender, age, pregnancy, mode)
  elif mode=="불편 현상에 따른 영양제 추천":
    prompt+="\n현재 사용자가 불편해 하는 상태를 해결할 수 있는 가장 적절한 영양소를 1가지 추천하세요.\n"
    nutrient_recommend(prompt,symptom, gender, age, pregnancy, mode)


def nutrient_recommend(condition,symptom, gender, age, pregnancy, mode):
    prompt = condition
    messages = [
        {
            "role": "system",
            "content": "너는 건강 기반의 영양제 조언 전문가야. 전문적으로 답변해줘."
        },
        {"role": "user", "content": prompt}
    ]

    # Step 1
    print("현재 messages:", messages)
    res = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        functions= [recommend_nutrient],
        function_call={"name": "recommend_nutrient"},
        temperature=0.5
        )
    # print("LLM 전체 응답:", res)
    func_name = res.choices[0].message.function_call.name
    print("호출된 함수:", func_name)
    print("함수 인자(raw):", res.choices[0].message.function_call.arguments)

    args = json.loads(res.choices[0].message.function_call.arguments)
    nutrient = args["nutrient"]
    print(f"추천 영양소: {nutrient}")

    # Step 2
    search_results = vectordb_search(args["query"])  # 실제 검색 실행
    messages.append({
      "role": "assistant",
      "content": None,
      "function_call": {
          "name": func_name,
          "arguments": res.choices[0].message.function_call.arguments
        }
    })

    messages.append({
        "role": "function",
        "name": func_name,
        "content": f"VectorDB 검색 결과: {json.dumps(search_results)}"
    })

    messages.append({
        "role": "user",
        "content": f"Vector DB에서 검색한 추천하는 영양소를 포함하는 영양제 제품 중 제일 적합한 영양제 제품 하나를 찾으세요"
    })

    res = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        functions= [vectordb_search_schema],
        function_call={"name": "vectordb_search"},
        temperature=0.5
        )

    func_name = res.choices[0].message.function_call.name
    print("호출된 함수:", func_name)
    print("함수 인자(raw):", res.choices[0].message.function_call.arguments)
    print("메세지: ",res.choices[0].message)

    # Step 3
    args = json.loads(res.choices[0].message.function_call.arguments)

    search_results = google_search(args["query"])  # 실제 검색 실행
    messages.append({
      "role": "assistant",
      "content": None,
      "function_call": {
          "name": func_name,
          "arguments": res.choices[0].message.function_call.arguments
        }
    })

    messages.append({
        "role": "function",
        "name": func_name,
        "content": f"Google 검색 결과: {json.dumps(search_results)}"
    })
    messages.append({
        "role": "user",
        "content": f"찾은 영양제 제품을 Google에서 검색하여 자세한 함량 정보를 수집하세요.",

    })
    res = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        functions=[google_search_function_schema],
        function_call={"name": "google_search"},
        temperature=0.5
    )
    func_name = res.choices[0].message.function_call.name
    print("호출된 함수:", func_name)
    print("함수 인자(raw):", res.choices[0].message.function_call.arguments)
    print("메세지: ",res.choices[0].message)

    # 최종 응답 생성 부분 수정
    final_response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages + [{
            "role": "user",
            "content": f"""
            지금까지의 검색 결과를 종합해서 다음 형식의 표로 정확하게 정리해주세요.
            | 제조사 | 제품명 | 주요 성분 | 함량 | 링크 |
            |--------|--------|-----------|------|------|
            필수 항목: 제조사, 제품명, 주요 영양 성분, 함량, 구매 링크

            """
        }]
    )


    print("\n최종 결과:")
    print(final_response.choices[0].message.content)



def llm_divider(symptom, gender, age, pregnancy, mode):
  if mode=="식습관 분석을 통한 영양제 추천" or mode=="불편 현상에 따른 영양제 추천" :
    nutrient_recommend_check(symptom, gender, age, pregnancy, mode)
  elif mode=="영양제 과다 복용 확인":
    nutrient_too_much_check(symptom, gender, age, pregnancy, mode)


# User Interface 정의 코드

In [21]:
import ipywidgets as widgets
from IPython.display import display, clear_output

# --- 위젯 UI 구성 ---
gender = widgets.ToggleButtons(options=["남성", "여성"], description='성별:')
age = widgets.IntText(value=30, description='만 나이:')
pregnancy = widgets.ToggleButtons(options=["해당 없음", "임신 중", "임신 가능성"], description='임신 여부:')
mode = widgets.ToggleButtons(options=["식습관 분석을 통한 영양제 추천", "불편 현상에 따른 영양제 추천", "영양제 과다 복용 확인"], description='메뉴:')

# ✅ 입력창 너비 확장
symptom_input = widgets.Text(
    placeholder='증상 또는 복용 제품 입력',
    description='입력:',
    layout=widgets.Layout(width='95%')
)

submit_button = widgets.Button(description="제출", button_style="success")
output_box = widgets.Output()

# --- 실행 로직 ---
def on_submit_clicked(b):
    with output_box:
        clear_output()
        info = {
            "성별": gender.value,
            "나이": age.value,
            "임신 여부": pregnancy.value,
            "선택 기능": mode.value
        }
        symptom = symptom_input.value.strip()

        print(f"사용자 정보: {info}")
        print(f"선택한 기능: {mode.value}")
        print()

        if not symptom:
            print("⚠️ 증상이나 복용 제품을 입력해주세요.")
            return

        # get_nutrient_action(symptom, gender.value, age.value, pregnancy.value, mode.value)
        llm_divider(symptom, gender.value, age.value, pregnancy.value, mode.value)
        print("\n ※ 이 답변은 빅데이터 기반으로 제공되며, 정확한 복약은 전문가 상담이 필요합니다.")

# User Interface 실행

In [22]:
# --- UI 연결 및 실행 ---
submit_button.on_click(on_submit_clicked)
display(widgets.VBox([
    gender, age, pregnancy, mode, symptom_input, submit_button, output_box
]))
